In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
# Set random seeds for reproducibility
torch.manual_seed(42)

In [ ]:
# Check for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

In [ ]:
df = pd.read_csv('/content/fashion-mnist_train.csv')
df.head()

In [ ]:
# train test split

X = df.iloc[:, 1:].values
y = df.iloc[:, 0].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
X_train = X_train/255.0
X_test = X_test/255.0

In [ ]:
class CustomDataset(Dataset):

  def __init__(self, features, labels):

    # Convert to PyTorch tensors
    self.features = torch.tensor(features, dtype=torch.float32)
    self.labels = torch.tensor(labels, dtype=torch.long)

  def __len__(self):
    return len(self.features)

  def __getitem__(self, index):
    return self.features[index], self.labels[index]

In [ ]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [ ]:
class MyNN(nn.Module):

  def __init__(self, input_dim, output_dim, num_hidden_layer,neurons_per_layer, dropout_rate):

    super().__init__()
    layers = []
    for i in range(num_hidden_layer): 
      layers.append(nn.Linear(input_dim,neurons_per_layer))
      layers.append(nn.BatchNorm1d(neurons_per_layer))
      layers.append(nn.ReLU())
      layers.append(nn.Dropout(dropout_rate))
      input_dim = neurons_per_layer

    layers.append(nn.Linear(neurons_per_layer,output_dim))
    
    self.model = nn.Sequential(*layers)
  
  def forward(self,x):
    return self.model(x)

In [ ]:
#objective function
def objective(trial): 
  # next hyperparamter values from the search space
  num_hidden_layer = trial.suggest_int('num_hidden_layer',1,5)
  neurons_per_layer = trial.suggest_int('neurons_per_layer',8,128,step=8)
  epochs = trial.suggest_int('epochs', 10,50)
  learning_rate = trial.suggest_float('learning_rate',1e-5,1e-1, log=True)
  dropout_rate = trial.suggest_float('drop',0.1,0.5, step = 0.1)
  batch_size = trial.suggest_categorical('batch_size',[16,32,64,128])
  optimizer_name = trial.suggest_categorical('optimizer',['Adam','SGD','RMSprop'])
  weight_decay = trial.suggest_float('weight_decay',1e-5, 1e-3 , log=True)

  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, pin_memory=True)   

  # model init
  input_dim = 784
  output_dim = 10
  model = MyNN(input_dim, output_dim, num_hidden_layer, neurons_per_layer, dropout_rate)
  model.to(device)

  # optimi5r selection
  criterion = nn.CrossEntropyLoss()
  # optimizer
  optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

  if optimizer_name == 'Adam':
    optim.Adam(model.parameters(), lr=learning_rate,weight_decay=weight_decay)

  elif optimizer_name == 'SGD':
    optim.SGD(model.parameters(), lr=learning_rate,weight_decay=weight_decay)

  else:
    optim.RMSprop(model.parameters(), lr=learning_rate,weight_decay=weight_decay)
  #training loop
  for epoch in range(epochs):

    total_epoch_loss = 0

    for batch_features, batch_labels in train_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)
      # forward pass
      outputs = model(batch_features)

      # calculate loss
      loss = criterion(outputs, batch_labels)

      # back pass
      optimizer.zero_grad()
      loss.backward()

      # update grads
      optimizer.step()
  # evaluation
  model.eval()
  # evaluation code
  total = 0
  correct = 0

  with torch.no_grad():

    for batch_features, batch_labels in test_loader:

      # move data to gpu
      batch_features, batch_labels = batch_features.to(device), batch_labels.to(device)

      outputs = model(batch_features)

      _, predicted = torch.max(outputs, 1)

      total = total + batch_labels.shape[0]

      correct = correct + (predicted == batch_labels).sum().item()

  accuracy = correct/total
  return accuracy
  

In [ ]:
import optuna
study = optuna.create_study(direction='maximize')

In [ ]:
study.optimize(objective, n_trials=10)

In [ ]:
study.best_value
study.best_params